In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Reading csv file
df = pd.read_csv("day.csv")

In [ ]:
# Displaying first 5 records of the dataset
df.head()

In [ ]:
# Checking number of rows and columns in the dataset
df.shape

In [ ]:
# Checking datatypes of each column in the dataset
df.info()

In [ ]:
# Checking for any null/missing values in the dataset
df.isnull().sum()

In [ ]:
# Summary of all numerical columns in the dataset
df.describe()

In [ ]:
# We can clearly see that instant is an index column so dropping this column
df.drop(['instant'], axis=1, inplace=True)
df.head()

In [ ]:
# From the data dictionary we know that cnt = casual + registered. Since our target variable is cnt, dropping columns
# casual and registered.
df.drop(['casual', 'registered'], axis=1, inplace=True)
df.head()

In [ ]:
# Dropping `dteday` as it is just the date format of the columns `weekday`-`mnth`-`yr`
df.drop(['dteday'], axis=1, inplace=True)
df.head()

In [ ]:
# Renaming the columns for better understanding
df.rename(columns = {'yr':'year','mnth':'month', 'atemp':'feelingtemp', 'hum':'humidity','cnt':'count'}, inplace = True) 
df.head()

In [ ]:
# Replacing categorical variables with their corresponding values referring to the data dictionary
df['season'] = df.season.map({1: 'spring', 2: 'summer', 3:'fall', 4:'winter'})
df['month'] = df.month.map({1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'June', 7:'July', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'})
df['weathersit'] = df.weathersit.map({1: 'Clear',2:'Mist + Cloudy',3:'Light Snow',4:'Heavy Rain + Ice Pallets + Thunderstorm + Mist'})
df['weekday'] = df.weekday.map({0:'Sun', 1:'Mon', 2:'Tue', 3:'Wed', 4:'Thu', 5:'Fri', 6:'Sat'})

df.head()

In [ ]:
# Visualise pairplot for numerical variables
sns.pairplot(df, vars=["temp", "humidity", "windspeed", "feelingtemp", "count"])
plt.show()

#### Few observations

1. `temp` and `feelingtemp` seems to be highly corelated to each other.
2. `windspeed` seems to be little positively correlated with count/

In [ ]:
plt.figure(figsize=(20, 10))

plt.subplot(2, 4, 1)
sns.boxplot(x = 'season', y = 'count', data = df)
plt.subplot(2, 4, 2)
sns.boxplot(x = 'year', y = 'count', data = df)
plt.subplot(2, 4, 3)
sns.boxplot(x = 'month', y = 'count', data = df)
plt.subplot(2, 4, 4)
sns.boxplot(x = 'weekday', y = 'count', data = df)
plt.subplot(2, 4, 5)
sns.boxplot(x = 'holiday', y = 'count', data = df)
plt.subplot(2, 4, 6)
sns.boxplot(x = 'weathersit', y = 'count', data = df)
plt.subplot(2, 4, 7)
sns.boxplot(x = 'workingday', y = 'count', data = df)
plt.show()

#### Observations
1. Bike rentals are more during fall season and then in summer.
2. Bike rentals in 2019 is more than in 2018. This might be due to bike sharing system is gaining popularity year after year.
3. Bike rentals are high in May to October which again falls under Fall and summer season.
4. Bike rentals demand is more on non-holidays. It might be due to people will commute more on non-holidays to offices, schools, universities etc.
5. Bike rentals is more on Clear, Few clouds, Partly cloudy, Partly cloudy days. And as expected there are no bikerentals on Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog days.
6. Working day or holiday doesn’t seem to have much effect on bike rentals.


In [ ]:
# Heatmap to check correlation between variables
plt.figure(figsize=(20, 12))
sns.heatmap(df.corr(), annot = True, cmap='YlGnBu')
plt.show()

#### Observations
`temp` and `atemp` (which is rename to feelingtemp) has very strong correlation with `count` which is a target variable.


## Dummy Variables

In [ ]:
# Create dummy variables for months, weekdays, weathersit, seasons

months_dummy = pd.get_dummies(df.month, drop_first = True)
seasons_dummy = pd.get_dummies(df.season, drop_first = True)
weekdays_dummy = pd.get_dummies(df.weekday, drop_first = True)
weathersit_dummy = pd.get_dummies(df.weathersit, drop_first = True)

In [ ]:
# Add these dummy datasets to the original dataset
df = pd.concat([months_dummy, weekdays_dummy, weathersit_dummy, seasons_dummy, df], axis = 1)
df.head()

In [ ]:
# Dropping `season`, `month`, `weekday`, `weathersit` as we have created dummy variables for it
df.drop(['month', 'season', 'weekday', 'weathersit'], axis = 1, inplace = True)
df.head()

In [ ]:
# Heatmap to check correlation between variables
plt.figure(figsize=(25, 12))
sns.heatmap(df.corr(), annot = True, cmap='YlGnBu')
plt.show()

In [ ]:
# it is clear from the above heatmap that variables `temp` and `feelingtemp` are strongly correlated. Hence dropping
# column `feelingtemp`

df.drop(['feelingtemp'], axis = 1, inplace = True)
df.head()

## Building the model into training and testing sets

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df_train, df_test = train_test_split(df, train_size = 0.7, random_state = 100)

In [ ]:
df_train.shape

In [ ]:
df_test.shape

### Rescaling the features

In [ ]:
# Instantiate an object
scaler = MinMaxScaler()

#create a list of numeric vars
num_vars = ['temp', 'humidity', 'windspeed', 'count']

# Fit on data
df_train[num_vars] = scaler.fit_transform(df_train[num_vars])
df_train.head()

In [ ]:
df_train[num_vars].describe()

In [ ]:
# Heatmap to check correlation between variables
plt.figure(figsize=(25, 12))
sns.heatmap(df.corr(), annot = True, cmap='YlGnBu')
plt.show()

#### Observation

We can observe that `temp`, `year` and `June` to `Oct` months seems to have good corelation with `count`

In [ ]:
# Dividing the tarining dataset into features and target variable
y_train = df_train.pop('count')
X_train = df_train

### Building linear regression model

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [ ]:
# Running RFE with output number of variables equal to 15

lm = LinearRegression()
lm.fit(X_train, y_train)

rfe = RFE(lm, 15)
rfe = rfe.fit(X_train, y_train)

In [ ]:
# Variables which are selected
list(zip(X_train.columns, rfe.support_, rfe.ranking_))

In [ ]:
# Features which are selected by RFE
X_train.columns[rfe.support_]

In [ ]:
# Features which are not selected by RFE
X_train.columns[~rfe.support_]

### Building model using statsmodel, for the detailed analysis

In [ ]:
# Creating X_test dataframe with RFE selected variables
X_train_rfe = X_train[X_train.columns[rfe.support_]]

In [ ]:
# Adding a constant variable
import statsmodels.api as sm  
import statsmodels.tsa.api as smt
X_train_rfe = sm.add_constant(X_train_rfe)

X_train_rfe.head()

In [ ]:
# Running the linear model 
lm = sm.OLS(y_train, X_train_rfe).fit()

In [ ]:
print(lm.summary())

In [ ]:
# Creating a dataframe which will contain names of all the features and its corresponding VIF values
# Calculating VIF for the above model

from statsmodels.stats.outliers_influence import variance_inflation_factor

X_train_curr = X_train_rfe
X_train_rfe = X_train_rfe.drop(['const'], axis=1)

vif = pd.DataFrame()
vif['Features'] = X_train_rfe.columns
vif['VIF'] = [variance_inflation_factor(X_train_rfe.values, i) for i in range(X_train_rfe.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = 'VIF', ascending = False)
vif

#### January column can be dropped as it have high p value and low VIF

In [ ]:
# Dropping January
X_train_v2 = X_train_rfe.drop(["Jan"], axis = 1)

In [ ]:
X_train_v2 = sm.add_constant(X_train_v2)
lm_v2 = sm.OLS(y_train, X_train_v2).fit()
print(lm_v2.summary())

In [ ]:
X_train_curr = X_train_v2
X_train_v2 = X_train_v2.drop(['const'], axis=1)

vif = pd.DataFrame()
vif['Features'] = X_train_v2.columns
vif['VIF'] = [variance_inflation_factor(X_train_v2.values, i) for i in range(X_train_v2.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = 'VIF', ascending = False)
vif

#### Now, all the p values are almost equal to 0, sp lets remove the feature having highest VIF i.e humidity

In [ ]:
# Dropping Humidity
X_train_v3 = X_train_v2.drop(["humidity"], axis = 1)

In [ ]:
X_train_v3 = sm.add_constant(X_train_v3)
lm_v3 = sm.OLS(y_train, X_train_v3).fit()
print(lm_v3.summary())

In [ ]:
X_train_curr = X_train_v3
X_train_v3 = X_train_v3.drop(['const'], axis=1)

vif = pd.DataFrame()
vif['Features'] = X_train_v3.columns
vif['VIF'] = [variance_inflation_factor(X_train_v3.values, i) for i in range(X_train_v3.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = 'VIF', ascending = False)
vif

#### Since temp has highest VIP value, we will check dropping this feature.

In [ ]:
# Dropping temp
X_train_v4 = X_train_v3.drop(["temp"], axis = 1)

In [ ]:
X_train_v4 = sm.add_constant(X_train_v4)
lm_v4 = sm.OLS(y_train, X_train_v4).fit()
print(lm_v4.summary())

#### Since there is a huge drop in Adjusted R^2, it is not good idea to drop `temp` feature. Hence rolling back to v3

#### Since workingday has next highest VIP value, we will check dropping this feature.

In [ ]:
# Dropping workingday
X_train_v5 = X_train_v3.drop(["workingday"], axis = 1)

In [ ]:
X_train_v5 = sm.add_constant(X_train_v5)
lm_v5 = sm.OLS(y_train, X_train_v5).fit()
print(lm_v5.summary())

In [ ]:
X_train_curr = X_train_v5
X_train_v5 = X_train_v5.drop(['const'], axis=1)

vif = pd.DataFrame()
vif['Features'] = X_train_v5.columns
vif['VIF'] = [variance_inflation_factor(X_train_v5.values, i) for i in range(X_train_v5.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = 'VIF', ascending = False)
vif

#### Since Sat has next highest p value (above 0.05) and low VIF, we will check dropping this feature.

In [ ]:
# Dropping Sat
X_train_v6 = X_train_v5.drop(["Sat"], axis = 1)

In [ ]:
X_train_v6 = sm.add_constant(X_train_v6)
lm_v6 = sm.OLS(y_train, X_train_v6).fit()
print(lm_v6.summary())

In [ ]:
X_train_curr = X_train_v6
X_train_v6 = X_train_v6.drop(['const'], axis=1)

vif = pd.DataFrame()
vif['Features'] = X_train_v6.columns
vif['VIF'] = [variance_inflation_factor(X_train_v6.values, i) for i in range(X_train_v6.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = 'VIF', ascending = False)
vif

#### Since July has next highest p value (equals 0.05) and low VIF, we will check dropping this feature.

In [ ]:
# Dropping July
X_train_v7 = X_train_v6.drop(["July"], axis = 1)

In [ ]:
X_train_v7 = sm.add_constant(X_train_v7)
lm_v7 = sm.OLS(y_train, X_train_v7).fit()
print(lm_v7.summary())

In [ ]:
X_train_curr = X_train_v7
X_train_v7 = X_train_v7.drop(['const'], axis=1)

vif = pd.DataFrame()
vif['Features'] = X_train_v7.columns
vif['VIF'] = [variance_inflation_factor(X_train_v7.values, i) for i in range(X_train_v7.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = 'VIF', ascending = False)
vif

### Residual Analysis

In [ ]:
y_train_pred = lm_v7.predict(X_train_curr)

In [ ]:
res = y_train - y_train_pred

plt.figure(figsize = (15, 8))
sns.distplot(res)
plt.show()

#### We can notice that the error terms are normally distributed

### Test for Homoscedasticity

In [ ]:
# plotting residual vs count scatter plot
sns.scatterplot(y_train, res)
plt.plot(y_train, [0]*len(y_train), '-r')
plt.xlabel('Count')
plt.ylabel('Residual')
plt.suptitle('Residual vs Count')
plt.show()

#### Observation:

We have created scatterplot with the residuals against the dependent variable. We can clearly observe that there is a constant deviation from the zero line and hence we can conclude our assumption of Homoscandasticity valid true.


### Test for Auto-correlation assumtion

In [ ]:
acf = smt.graphics.plot_acf(res, lags=40 , alpha=0.05)
acf.show()

#### Observation:

Since there are no much error components crossing the greyed out area (confidence interval) and hence we can say that there is no pattern in the error and hence we can say No auto-correlation assumption has been preserved.

### Predictions and Evaluation of the test set

In [ ]:
#create a list of numeric vars
num_vars = ['temp', 'humidity', 'windspeed', 'count']

# Fit on data
df_test[num_vars] = scaler.transform(df_test[num_vars])
df_test.head()

In [ ]:
df_test.describe()

In [ ]:
# Dividing the testing dataset into features and target variable
y_test = df_test.pop('count')
X_test = df_test

In [ ]:
X_train_v7.columns

In [ ]:
X_test_new = X_test[X_train_v7.columns]
X_test_new = sm.add_constant(X_test_new)
X_test_new.head()

In [ ]:
# Prediction for testing data
y_test_pred = lm_v7.predict(X_test_new)

In [ ]:
# Model Evaluation
from sklearn.metrics import r2_score
r2_score(y_test, y_test_pred)

In [ ]:
# Adjusted R^2
# Adj R^2 = 1-(1-R^2)*(n-1)/(n-p-1))

# n = sample size , p = number of independent variables

adj_r2 = 1-((1-0.8038195990728842)*(220-1)/(220-10-1))
print(adj_r2)

### Equation of the best fit line is

*count = 0.0910 x Sept - 0.2850 x Light Snow - 0.0787 x (Mist + Cloudy) - 0.0554 x spring + 0.0621 x summer + 0.0945 x winter + 0.2341 x year - 0.0963 x holiday + 0.4777 x temp - 0.1481 x windspeed + 0.1909*